In [12]:
import os 
from pathlib import Path
import sys
import subprocess
import time
from typing import Optional
from collections import defaultdict 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
from IPython.display import display
import latextable
from texttable import Texttable

In [71]:
input_data = """
benchmark    |  genmc   |  genmc  |  hmc     |  hmc   |  cdschecker  |  cdschecker  |  wmc   |  wmc   |  xmm   |  xmm   |
reorder2/    |  1296    |  .10    |  1296    |  .09   |  503         |  0.03        |  1296  |  .05   |  1296  |  .21   |
singleton/   |  4       |  .05    |  24      |  .04   |  4           |  0.00        |  24    |  .01   |  4     |  .05   |
fib_bench/   |  465696  |  4.32   |  465696  |  5.67  |  *           |  *           |  *     |  *     |  *     |  *     |
szymanski/1  |  6       |  .05    |  6       |  .05   |  560         |  0.24        |  444   |  .06   |  +     |  +     |
szymanski/2  |  78      |  .21    |  78      |  .43   |  *           |  *           |  *     |  *     |  +     |  +     |
szymanski/3  |  1068    |  2.32   |  1068    |  6.74  |  *           |  *           |  *     |  *     |  +     |  +     |
peterson/10  |  39      |  .04    |  40      |  .05   |  306         |  0.03        |  1152  |  .29   |  53    |  .05   |
peterson/20  |  39      |  .04    |  40      |  .06   |  1006        |  0.13        |  3892  |  2.25  |  53    |  .04   |
peterson/30  |  39      |  .04    |  40      |  .05   |  2106        |  0.32        |  8232  |  9.34  |  53    |  .05   |
dekker-bnd/  |  55      |  .05    |  55      |  .05   |  244         |  0.15        |  55    |  .01   |  61    |  .06   |
sigma/       |  *       |  *      |  *       |  *     |  *           |  *           |  *     |  *     |  *     |  *     |
indexer/12   |  8       |  .05    |  8       |  .05   |  8           |  0.60        |  8     |  .02   |  8     |  .05   |
indexer/13   |  64      |  .10    |  64      |  .16   |  *           |  *           |  64    |  .05   |  64    |  .13   |
indexer/14   |  512     |  .51    |  512     |  1.14  |  *           |  *           |  512   |  .30   |  512   |  .62   |
indexer/15   |  4096    |  4.47   |  *       |  *     |  *           |  *           |  4096  |  3.16  |  4096  |  6.03  |
"""

In [72]:
def un_underscore(name: str) -> str:
    return name.replace("_", "\\_")

def parse_name(name: str) -> str:
    match un_underscore(name).split('/'):
        case [a]:
            return a
        case [a, ""]:
            return a
        case [a, number]:
            return f"{a}({number})"

def parse_result(r: str) -> str:
    match r:
        case '*':
            return '{\\fontspec{Symbola}\\symbol{"231B}}'
        case "+":
            return ''
        case r:
            return r

def parse_line(line: str) -> tuple[list[str], list[str]]:
    parts = [p.strip() for p in line.split('|') if p != ""]
    name = parse_name(parts[0])
    parts = [parse_result(p) for p in parts[1:]]
    execs = [name] + [parts[i] for i in range(0, len(parts), 2)]
    times = [name] + [parts[i] for i in range(1, len(parts), 2)]
    return execs, times

lines = [l for l in input_data.splitlines() if l != ""][1:]
tuples = [parse_line(line) for line in lines]
execs = [['Test Name', 'GenMC execs', ' HMC execs', 'CDSCheker execs', 'WMC execs', 'XMM execs']] + [exe for exe, time in tuples]
times = [['Test Name', 'GenMC time', 'HMC time', 'CDSChecker time', 'WMC time', 'XMM time']] + [time for exe, time in tuples]

In [73]:
# max_len = [max([len(row[i]) for row in rows]) for i in range(len(rows[0]))]

table_1 = Texttable()
# table_1.set_cols_width(max_len)
table_1.set_cols_align(["l"] + ["c" for i in range(len(execs[0]) - 1)])
table_1.set_cols_valign(["m" for i in range(len(execs[0]))])
table_1.add_rows(execs)
# print(table_1.draw())
# print('\nLatextable Output:')
print(latextable.draw_latex(table_1, position='H', caption="Number of executions on synthetic benchmarks run with GenMC, HMC, CDSChecker, WMC, and GenMC-XMM", label="table:synthetic-benchmarks-execs"))

\begin{table}
	\begin{center}
		\begin{tabular}{|l|c|c|c|c|c|}
			\hline
			Test Name & GenMC execs &  HMC execs & CDSCheker execs & WMC execs & XMM execs \\
			\hline
			reorder2 & 1296 & 1296 & 503 & 1296 & 1296 \\
			\hline
			singleton & 4 & 24 & 4 & 24 & 4 \\
			\hline
			fib\_bench & 465696 & 465696 & {\fontspec{Symbola}\symbol{"231B}} & {\fontspec{Symbola}\symbol{"231B}} & {\fontspec{Symbola}\symbol{"231B}} \\
			\hline
			szymanski(1) & 6 & 6 & 560 & 444 &  \\
			\hline
			szymanski(2) & 78 & 78 & {\fontspec{Symbola}\symbol{"231B}} & {\fontspec{Symbola}\symbol{"231B}} &  \\
			\hline
			szymanski(3) & 1068 & 1068 & {\fontspec{Symbola}\symbol{"231B}} & {\fontspec{Symbola}\symbol{"231B}} &  \\
			\hline
			peterson(10) & 39 & 40 & 306 & 1152 & 53 \\
			\hline
			peterson(20) & 39 & 40 & 1006 & 3892 & 53 \\
			\hline
			peterson(30) & 39 & 40 & 2106 & 8232 & 53 \\
			\hline
			dekker-bnd & 55 & 55 & 244 & 55 & 61 \\
			\hline
			sigma & {\fontspec{Symbola}\symbol{"231B}} & {\fonts

In [74]:
table_1 = Texttable()
table_1.set_precision(2)
table_1.set_cols_align(["l"] + ["c" for i in range(len(times[0]) - 1)])
table_1.set_cols_valign(["m" for i in range(len(times[0]))])
table_1.add_rows(times)
# print(table_1.draw())
# print('\nLatextable Output:')
print(latextable.draw_latex(table_1, position='H', caption="Execution time (seconds) on synthetic benchmarks run with GenMC, HMC, CDSChecker, WMC, and GenMC-XMM", label="table:synthetic-benchmarks-time"))

\begin{table}
	\begin{center}
		\begin{tabular}{|l|c|c|c|c|c|}
			\hline
			Test Name & GenMC time & HMC time & CDSChecker time & WMC time & XMM time \\
			\hline
			reorder2 & 0.10 & 0.09 & 0.03 & 0.05 & 0.21 \\
			\hline
			singleton & 0.05 & 0.04 & 0 & 0.01 & 0.05 \\
			\hline
			fib\_bench & 4.32 & 5.67 & {\fontspec{Symbola}\symbol{"231B}} & {\fontspec{Symbola}\symbol{"231B}} & {\fontspec{Symbola}\symbol{"231B}} \\
			\hline
			szymanski(1) & 0.05 & 0.05 & 0.24 & 0.06 &  \\
			\hline
			szymanski(2) & 0.21 & 0.43 & {\fontspec{Symbola}\symbol{"231B}} & {\fontspec{Symbola}\symbol{"231B}} &  \\
			\hline
			szymanski(3) & 2.32 & 6.74 & {\fontspec{Symbola}\symbol{"231B}} & {\fontspec{Symbola}\symbol{"231B}} &  \\
			\hline
			peterson(10) & 0.04 & 0.05 & 0.03 & 0.29 & 0.05 \\
			\hline
			peterson(20) & 0.04 & 0.06 & 0.13 & 2.25 & 0.04 \\
			\hline
			peterson(30) & 0.04 & 0.05 & 0.32 & 9.34 & 0.05 \\
			\hline
			dekker-bnd & 0.05 & 0.05 & 0.15 & 0.01 & 0.06 \\
			\hline
			sigma & {\